In [28]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
url = 'https://sportsdatabase.com/nba/query?output=default&sdql=date%2Cteam%2Co%3Ateam%2Csite%2Cfield+goals+attempted%2Cfield+goals+made%2Cfree+throws+attempted%2Cfree+throws+made%2Cthree+pointers+attempted%2Cthree+pointers+made%2Cassists%2Cblocks%2Crebounds%2Csteals%2Cturnovers%2Cline%2Cmargin%2Ctotal%2Cou+margin%2Cstreak+%40season+%3D+2019&submit=++S+D+Q+L+%21++'
my_header = { "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36", "X-Requested-With": "XMLHttpRequest" }
html = requests.get(url, headers=my_header)
df = pd.read_html(html.text)

In [266]:
df = pd.read_csv('NBA2019.csv')

In [267]:
df

,date,team,o:team,site,field goals attempted,field goals made,field goal %,free throws attempted,free throws made,free throw %,...,rebounds,steals,turnovers,line,margin,ats outcome,total,ou margin,ou outcome,streak
0,20191022,Clippers,Lakers,home,81,42,51.851852,24,17,70.833333,...,45,8,14,1.5,10,1,224.0,-10.0,U,0
1,20191022,Lakers,Clippers,away,85,37,43.529412,21,15,71.428571,...,41,4,14,-1.5,-10,0,224.0,-10.0,U,0
2,20191022,Raptors,Pelicans,home,103,42,40.776699,38,32,84.210526,...,57,7,16,-7.0,8,1,231.5,20.5,O,0
3,20191022,Pelicans,Raptors,away,102,43,42.156863,20,17,85.000000,...,53,4,19,7.0,-8,0,231.5,20.5,O,0
4,20191023,Heat,Grizzlies,home,85,40,47.058824,39,33,84.615385,...,51,9,21,-8.0,19,1,214.0,7.0,O,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2281,20201006,Lakers,Heat,away,79,35,44.303797,21,18,85.714286,...,42,5,15,-7.5,6,1,218.5,-20.5,U,-1
2282,20201009,Lakers,Heat,home,82,38,46.341463,21,18,85.714286,...,41,10,15,-7.0,-3,1,217.0,2.0,O,1
2283,20201009,Heat,Lakers,away,83,38,45.783133,22,21,95.454545,...,35,7,13,7.0,3,0,217.0,2.0,O,-1
2284,20201011,Heat,Lakers,home,79,35,44.303797,22,13,59.090909,...,41,4,13,6.0,-13,0,214.5,-15.5,U,1


In [7]:
fg = []
ft = []
tp = []
ats = []
ou = []
for i in range(len(df)):
    fg.append(int(df['field goals made'][i])/int(df['field goals attempted'][i])*100)
    ft.append(int(df['free throws made'][i])/int(df['free throws attempted'][i])*100)
    tp.append(int(df['three pointers made'][i])/int(df['three pointers attempted'][i])*100)
    if df['margin'][i]>df['line'][i]:
        ats.append(1)
    else:
        ats.append(0)
    if df['ou margin'][i]>0:
        ou.append('O')
    elif df['ou margin'][i]<0:
        ou.append('U')
    else:
        ou.append('P')
# Computed stats were manually inserted into desired position in cell below

In [25]:
df.insert(23, 'ou outcome', ou)

In [16]:
def cummean(x):
    st = []
    stats = ['field goals attempted', 'field goals made', 'field goal %', 'free throws attempted', 'free throws made', 'free throw %', 'three pointers attempted', 'three pointers made', 'three point %', 'assists', 'blocks', 'rebounds', 'steals', 'turnovers']
    for col in stats:
        cumav = np.array(np.cumsum(x[col])/(np.arange(len(x), dtype=float)+1))
        cumav = list(np.delete(np.insert(cumav, 0, np.NaN), -1))
        st.append(cumav)
    return pd.DataFrame(np.transpose(np.array(st)), columns = stats)

In [274]:
# Groupby and calculate team average and stats
team_data = df.groupby('team')
training_data = pd.DataFrame()
for _, group in tqdm(team_data):
    wins = [0]
    losses = [0]
    for i in range(1,len(group)):
        if group.reset_index()['margin'][i] > 0:
            wins.append(wins[-1] + 1)
            losses.append(losses[-1])
        if group.reset_index()['margin'][i] < 0:
            wins.append(wins[-1])
            losses.append(losses[-1] + 1)
# Recombine after adding columns
    team_stats = pd.concat([group.reset_index()[['index', 'date', 'team', 'o:team', 'site']], cummean(group), group.reset_index()[['line', 'margin', 'ats outcome', 'total', 'ou margin', 'ou outcome', 'streak']]], axis = 1)
    team_stats.insert(5, 'wins', wins)
    team_stats.insert(6, 'losses', losses)
    training_data = pd.concat([training_data, team_stats], ignore_index = True)

100%|██████████| 30/30 [00:02<00:00, 10.98it/s]


In [275]:
training_data

,index,date,team,o:team,site,wins,losses,field goals attempted,field goals made,field goal %,...,rebounds,steals,turnovers,line,margin,ats outcome,total,ou margin,ou outcome,streak
0,29,20191024,Bucks,Rockets,away,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,1.5,6,1,230.0,-2.0,U,0
1,50,20191026,Bucks,Heat,home,0,1,99.000000,46.000000,46.464646,...,53.000000,7.000000,11.000000,-10.5,-5,1,226.5,30.5,O,1
2,80,20191028,Bucks,Cavaliers,home,1,1,96.500000,43.500000,45.040834,...,50.000000,7.500000,14.500000,-15.5,17,1,224.0,17.0,O,-1
3,111,20191030,Bucks,Celtics,away,1,2,95.000000,45.000000,47.418527,...,50.000000,8.000000,13.666667,-2.5,-11,0,222.5,-1.5,U,1
4,143,20191101,Bucks,Magic,away,2,2,91.750000,43.250000,47.149261,...,48.750000,6.750000,14.000000,-4.5,32,1,215.0,-1.0,U,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2281,2014,20200805,Wizards,Seventysixers,home,24,43,90.955224,41.865672,46.158734,...,41.731343,8.044776,13.477612,10.5,-9,0,229.0,-24.0,U,-3
2282,2033,20200807,Wizards,Pelicans,away,24,44,90.867647,41.779412,46.102767,...,41.823529,8.044118,13.500000,8.5,-11,0,236.0,-11.0,U,-4
2283,2061,20200809,Wizards,Thunder,away,24,45,90.855072,41.724638,46.046527,...,41.797101,8.057971,13.492754,9.5,-18,0,222.0,2.0,O,-5
2284,2086,20200811,Wizards,Bucks,home,24,46,90.871429,41.671429,45.978782,...,41.828571,8.028571,13.514286,7.0,-13,0,227.0,12.0,O,-6


In [276]:
training_data = training_data.sort_values(by = ['index']).reset_index(drop=True)
training_data.drop(list(range(32)), inplace=True)
training_data.reset_index(drop=True, inplace=True)
training_data = training_data[['index', 'date', 'team', 'o:team', 'site', 'wins', 'losses','field goals attempted', 'field goals made', 'field goal %', 'free throws attempted', 'free throws made', 'free throw %', 'three pointers attempted', 'three pointers made', 'three point %', 'assists', 'blocks', 'rebounds', 'steals', 'turnovers', 'streak', 'line', 'margin', 'ats outcome', 'total', 'ou margin', 'ou outcome']]

In [277]:
training_data

,index,date,team,o:team,site,wins,losses,field goals attempted,field goals made,field goal %,...,rebounds,steals,turnovers,streak,line,margin,ats outcome,total,ou margin,ou outcome
0,32,20191025,Celtics,Raptors,home,1,0,90.000000,33.000000,36.666667,...,41.000000,4.000000,10.000000,-1,-3.0,6,1,216.5,1.5,O
1,33,20191025,Raptors,Celtics,away,0,1,103.000000,42.000000,40.776699,...,57.000000,7.000000,16.000000,1,3.0,-6,0,216.5,1.5,O
2,34,20191025,Grizzlies,Bulls,home,0,1,89.000000,38.000000,42.696629,...,40.000000,9.000000,24.000000,-1,2.0,-8,0,222.5,-10.5,U
3,35,20191025,Bulls,Grizzlies,away,1,0,105.000000,49.000000,46.666667,...,49.000000,11.000000,10.000000,-1,-2.0,8,1,222.5,-10.5,U
4,36,20191025,Hornets,Timberwolves,home,0,1,88.000000,45.000000,51.136364,...,41.000000,3.000000,19.000000,1,5.5,-22,0,226.5,-6.5,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2249,2281,20201006,Lakers,Heat,away,67,22,87.449438,42.157303,48.219376,...,45.438202,8.539326,14.617978,-1,-7.5,6,1,218.5,-20.5,U
2250,2282,20201009,Lakers,Heat,home,67,23,87.355556,42.077778,48.175870,...,45.400000,8.500000,14.622222,1,-7.0,-3,1,217.0,2.0,O
2251,2283,20201009,Heat,Lakers,away,57,35,83.945652,39.119565,46.731493,...,43.793478,7.413043,13.597826,-1,7.0,3,0,217.0,2.0,O
2252,2284,20201011,Heat,Lakers,home,57,36,83.935484,39.107527,46.721296,...,43.698925,7.408602,13.591398,1,6.0,-13,0,214.5,-15.5,U


In [278]:
col = ['wins', 'losses', 'field goals attempted', 'field goals made', 'field goal %', 'free throws attempted', 'free throws made', 'free throw %', 'three pointers attempted', 'three pointers made', 'three point %', 'assists', 'blocks', 'rebounds', 'steals', 'turnovers', 'streak']
icol = ['o:wins','o:losses','o:field goals attempted', 'o:field goals made', 'o:field goal %', 'o:free throws attempted', 'o:free throws made', 'o:free throw %', 'o:three pointers attempted', 'o:three pointers made', 'o:three point %', 'o:assists', 'o:blocks', 'o:rebounds', 'o:steals', 'o:turnovers', 'o:streak']
opposing_stats = []
for i in range(0, len(training_data), 2):
    opposing_stats.append(list(training_data[col].loc[i+1]))
    opposing_stats.append(list(training_data[col].loc[i]))
training_data = pd.concat([training_data[['index', 'date', 'team', 'o:team', 'site','wins', 'losses', 'field goals attempted', 'field goals made', 'field goal %', 'free throws attempted', 'free throws made', 'free throw %', 'three pointers attempted', 'three pointers made', 'three point %', 'assists', 'blocks', 'rebounds', 'steals', 'turnovers', 'streak']], pd.DataFrame(np.array(opposing_stats), columns = icol), training_data[['line', 'margin', 'ats outcome', 'total', 'ou margin', 'ou outcome']]], axis = 1)

In [293]:
training_data[['wins', 'losses', 'field goals attempted', 'field goals made', 'field goal %', 'free throws attempted', 'free throws made', 'free throw %', 'three pointers attempted', 'three pointers made', 'three point %', 'assists', 'blocks', 'rebounds', 'steals', 'turnovers', 'streak', 'margin']].corr()

,wins,losses,field goals attempted,field goals made,field goal %,free throws attempted,free throws made,free throw %,three pointers attempted,three pointers made,three point %,assists,blocks,rebounds,steals,turnovers,streak,margin
wins,1.000000,0.412268,-0.100601,0.312530,0.454825,-0.014514,0.026528,0.104408,0.116394,0.220700,0.250797,0.200165,0.123064,0.131144,-0.024855,-0.403859,0.208546,0.125658
losses,0.412268,1.000000,-0.020041,-0.050600,-0.035836,-0.219848,-0.175294,0.065661,-0.013750,-0.007950,0.008182,0.018919,-0.217489,-0.329351,-0.087882,-0.285337,-0.240643,-0.167545
field goals attempted,-0.100601,-0.020041,1.000000,0.552606,-0.176467,-0.037956,-0.012374,0.045184,0.380375,0.213554,-0.251357,0.215297,0.245311,0.355074,0.122608,-0.182106,-0.029367,-0.003484
field goals made,0.312530,-0.050600,0.552606,1.000000,0.721180,-0.072148,-0.039042,0.065476,0.157300,0.251983,0.259642,0.546865,0.356520,0.304354,-0.029955,-0.156273,0.224307,0.098444
field goal %,0.454825,-0.035836,-0.176467,0.721180,1.000000,-0.057877,-0.033114,0.057204,-0.128868,0.123900,0.521937,0.458556,0.208920,0.057582,-0.141967,-0.036095,0.287362,0.118251
free throws attempted,-0.014514,-0.219848,-0.037956,-0.072148,-0.057877,1.000000,0.910073,-0.036697,0.317645,0.242841,-0.039745,-0.169540,0.054592,0.401908,0.074990,0.260104,0.075507,0.058043
free throws made,0.026528,-0.175294,-0.012374,-0.039042,-0.033114,0.910073,1.000000,0.372666,0.272476,0.214949,-0.027360,-0.134312,-0.010750,0.288929,0.028188,0.116773,0.082832,0.061397
free throw %,0.104408,0.065661,0.045184,0.065476,0.057204,-0.036697,0.372666,1.000000,-0.062304,-0.017776,0.058699,0.021383,-0.146080,-0.219831,-0.129579,-0.297666,0.036636,0.021197
three pointers attempted,0.116394,-0.013750,0.380375,0.157300,-0.128868,0.317645,0.272476,-0.062304,1.000000,0.888760,0.029413,0.020229,-0.018422,0.322461,0.019586,0.056263,0.046825,0.056169
three pointers made,0.220700,-0.007950,0.213554,0.251983,0.123900,0.242841,0.214949,-0.017776,0.888760,1.000000,0.476490,0.114979,-0.058319,0.251367,-0.114755,0.057443,0.116777,0.071319


In [279]:
training_data.loc[0]

index                               32
date                          20191025
team                           Celtics
o:team                         Raptors
site                              home
wins                                 1
losses                               0
field goals attempted               90
field goals made                    33
field goal %                   36.6667
free throws attempted               34
free throws made                    20
free throw %                   58.8235
three pointers attempted            26
three pointers made                  7
three point %                  26.9231
assists                             18
blocks                               2
rebounds                            41
steals                               4
turnovers                           10
streak                              -1
o:wins                               0
o:losses                             1
o:field goals attempted            103
o:field goals made       

In [280]:
training_data.loc[1]

index                               33
date                          20191025
team                           Raptors
o:team                         Celtics
site                              away
wins                                 0
losses                               1
field goals attempted              103
field goals made                    42
field goal %                   40.7767
free throws attempted               38
free throws made                    32
free throw %                   84.2105
three pointers attempted            40
three pointers made                 14
three point %                       35
assists                             23
blocks                               3
rebounds                            57
steals                               7
turnovers                           16
streak                               1
o:wins                               1
o:losses                             0
o:field goals attempted             90
o:field goals made       

In [ ]:
#Save to csv
training_data.sort_values(by = ['date']).drop(columns = ['index']).reset_index(drop=True).to_csv('NBA2019_Cumulative_Stats.csv', index = False)

In [19]:
cummean(team_data.get_group('Bucks'))

,field goals attempted,field goals made,field goal %,free throws attempted,free throws made,free throw %,three pointers attempted,three pointers made,three point %,assists,blocks,rebounds,steals,turnovers
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,99.000000,46.000000,46.464646,18.000000,9.000000,50.000000,46.000000,16.000000,34.782609,31.000000,10.000000,53.000000,7.000000,11.000000
2,96.500000,43.500000,45.040834,26.500000,18.000000,63.571429,50.000000,16.500000,33.132045,26.500000,7.000000,50.000000,7.500000,14.500000
3,95.000000,45.000000,47.418527,25.666667,17.333333,64.603175,46.000000,16.666667,37.000311,27.333333,7.000000,50.000000,8.000000,13.666667
4,91.750000,43.250000,47.149261,25.250000,16.750000,64.077381,45.750000,16.000000,35.528011,25.750000,6.250000,48.750000,6.750000,14.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,90.769231,43.230769,47.722941,24.474359,18.128205,73.592187,39.051282,13.923077,35.729577,25.974359,5.807692,51.653846,7.166667,14.717949
79,90.569620,43.151899,47.743326,24.493671,18.075949,73.342237,39.000000,13.949367,35.855966,25.924051,5.822785,51.430380,7.151899,14.772152
80,90.500000,43.075000,47.690652,24.675000,18.262500,73.483151,38.825000,13.862500,35.757766,25.800000,5.775000,51.412500,7.137500,14.762500
81,90.469136,43.000000,47.620958,24.629630,18.222222,73.457786,38.802469,13.827160,35.683346,25.765432,5.765432,51.320988,7.148148,14.703704


In [6]:
#Imputation of Missing Data for a single game
df['field goals attempted'][1320] = '73'
df['field goals made'][1320] = '39'
df['free throws attempted'][1320] = '44'
df['free throws made'][1320] = '37'
df['three pointers attempted'][1320] = '29'
df['three pointers made'][1320] = '12'
df['assists'][1320] = '19'
df['blocks'][1320] = '4'
df['rebounds'][1320] = '53'
df['steals'][1320] = '9'
df['turnovers'][1320] = '17'
df['line'][1320] = np.int64(1)

df['field goals attempted'][1321] = '98'
df['field goals made'][1321] = '42'
df['free throws attempted'][1321] = '23'
df['free throws made'][1321] = '12'
df['three pointers attempted'][1321] = '35'
df['three pointers made'][1321] = '10'
df['assists'][1321] = '23'
df['blocks'][1321] = '1'
df['rebounds'][1321] = '55'
df['steals'][1321] = '10'
df['turnovers'][1321] = '11'
df['line'][1321] = np.int64(-1)






C:\Users\Adi\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Adi\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Adi\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can 

In [303]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn import metrics
#'field goals attempted', 'field goals made', 'free throws attempted', 'free throws made', 'three pointers attempted', 'three pointers made', 'o:field goals attempted', 'o:field goals made', 'o:free throws attempted', 'o:free throws made', 'o:three pointers attempted', 'o:three pointers made',
X = training_data[['wins', 'losses', 'field goal %','free throw %', 'three point %', 'assists', 'blocks', 'rebounds', 'steals', 'turnovers', 'streak','o:wins', 'o:losses', 'o:field goal %', 'o:free throw %',  'o:three point %', 'o:assists', 'o:blocks', 'o:rebounds', 'o:steals', 'o:turnovers', 'o:streak', 'line']]
y = training_data['ats outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 25)

In [304]:
rfcl = RandomForestClassifier(random_state = 25)
rfcl.fit(X_train, y_train)
svmcl = svm.SVC()
svmcl.fit(X_train, y_train)
y_pred1=rfcl.predict(X_test)
y_pred2=svmcl.predict(X_test)

In [305]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [50, 75, 100, 125, 150]
criterion = ['gini', 'entropy']
max_depth = [25, 50, 75, 100, 125, 150]
min_samples_split = [2, 3, 4]
min_samples_leaf = [1,2,3,4]
max_features = ['auto', 'sqrt', 'log2']
bootstrap = [True, False]

C = [0.25 ,0.5, 1, 1.5, 2]
kernel = ['linear', 'poly', 'rbf', 'sigmoid']
degree = [2,3,4,5]
gamma = ['auto', 'scale']
coef0 = [-2,-1,0, 1, 2]
tol = [5e-4, 1e-3, 1.5e-3, 2e-3]

rf_random_grid = {'n_estimators' : n_estimators,
               'criterion' : criterion,
              'max_depth' : max_depth,
              'min_samples_split' : min_samples_split,
              'min_samples_leaf' : min_samples_leaf,
              'max_features' : max_features,
              'bootstrap' : bootstrap}

svm_random_grid = {'C' : C,
                  'kernel': kernel,
                  'degree' : degree,
                  'gamma' : gamma,
                  'coef0' : coef0,
                  'tol' : tol}

rf_random = RandomizedSearchCV(estimator = rfcl, param_distributions = rf_random_grid, n_iter = 200, cv = 4, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)

Fitting 4 folds for each of 200 candidates, totalling 800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   13.7s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:   37.3s
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed:  1.5min finished


RandomizedSearchCV(cv=4, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [ ]:
svm_random = RandomizedSearchCV(estimator = svmcl, param_distributions = svm_random_grid, n_iter = 200, cv = 4, verbose=2, random_state=42, n_jobs = -1)
svm_random.fit(X_train, y_train)

Fitting 4 folds for each of 200 candidates, totalling 800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   15.8s


In [302]:
rf_best_random = rf_random.best_estimator_
svm_best_random = svm_random.best_estimator_
y_pred3 = rf_best_random.predict(X_test)
y_pred4 = svm_best_random.predict(X_test)
print("RF Accuracy:", metrics.accuracy_score(y_test, y_pred3))
print("SVM Accuracy:", metrics.accuracy_score(y_test, y_pred4))

Accuracy: 0.7272727272727273


In [301]:
print("RF Accuracy:",metrics.accuracy_score(y_test, y_pred1))
print("SVM Accuracy:",metrics.accuracy_score(y_test, y_pred2))


RF Accuracy: 0.7450110864745011
SVM Accuracy: 0.7583148558758315


In [297]:
weights = pd.Series(rfcl.feature_importances_,index=X_train.columns).sort_values(ascending=False)
weights

losses                   0.060029
o:losses                 0.058647
wins                     0.048785
o:wins                   0.046359
o:rebounds               0.041867
o:field goal %           0.041704
rebounds                 0.040027
blocks                   0.038423
field goal %             0.035996
o:three point %          0.035128
o:field goals made       0.034868
three point %            0.033966
free throws made         0.033955
field goals made         0.033429
o:blocks                 0.033381
o:turnovers              0.032712
assists                  0.032118
o:assists                0.031527
o:free throw %           0.031102
o:free throws made       0.031101
turnovers                0.031010
free throw %             0.029884
o:three pointers made    0.029760
o:steals                 0.029575
three pointers made      0.028198
steals                   0.027831
streak                   0.024503
o:streak                 0.024115
dtype: float64

In [288]:
X_test[0:25]

,wins,losses,field goal %,free throw %,three point %,assists,blocks,rebounds,steals,turnovers,...,o:losses,o:field goal %,o:free throw %,o:three point %,o:assists,o:blocks,o:rebounds,o:steals,o:turnovers,o:streak
1050,16,20,43.225873,76.353808,33.124738,22.805556,6.416667,44.444444,8.472222,12.055556,...,18.0,44.140213,74.889928,33.600357,23.764706,5.000000,48.470588,6.558824,15.294118,-5.0
2077,42,32,46.221057,77.802386,36.758479,24.756757,4.851351,47.000000,6.175676,12.175676,...,39.0,46.729465,83.103735,35.677460,27.125000,3.972222,43.555556,7.652778,14.333333,7.0
2149,46,29,45.114787,79.501108,34.615158,21.560000,5.066667,44.093333,8.666667,13.960000,...,29.0,46.824696,79.742590,35.385818,21.586667,4.880000,43.160000,7.493333,13.013333,1.0
675,13,8,44.710632,73.582478,34.906258,26.047619,4.952381,46.238095,7.666667,13.142857,...,7.0,47.731558,73.607261,37.208585,26.958333,5.791667,46.208333,8.916667,15.750000,2.0
629,15,7,47.438059,81.609287,36.399844,24.772727,5.227273,44.181818,6.954545,13.318182,...,18.0,42.262805,67.630570,36.318130,19.772727,4.727273,44.136364,7.818182,14.818182,-8.0
448,6,11,42.929941,75.005564,34.481813,23.058824,4.000000,44.529412,9.705882,14.588235,...,11.0,44.566659,80.301835,36.927769,19.588235,6.117647,45.647059,5.941176,13.882353,-4.0
343,2,11,43.453310,84.012624,31.698711,24.000000,3.923077,44.307692,7.615385,14.615385,...,8.0,46.328171,71.159008,38.646507,26.416667,4.750000,45.250000,9.250000,17.500000,-1.0
1782,41,18,46.331486,80.339913,36.723227,23.033898,5.559322,45.966102,8.288136,13.084746,...,32.0,44.524555,74.457180,34.589423,24.152542,4.610169,48.305085,6.576271,14.711864,-4.0
1908,38,26,46.475295,74.456005,36.044663,25.843750,5.406250,45.421875,8.187500,13.406250,...,46.0,45.923748,74.542098,36.912419,24.061538,4.538462,41.938462,7.384615,14.661538,-4.0
2104,50,23,46.126004,80.394753,36.526757,22.972603,5.602740,46.000000,8.369863,13.054795,...,32.0,46.776693,74.885944,36.638976,25.770270,5.310811,45.527027,7.986486,13.540541,-1.0


In [235]:
y_pred[0:25]

array([1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 0], dtype=int64)

In [236]:
y_test[0:25]

1050    1
2077    0
2149    0
675     0
629     1
448     0
343     0
1782    0
1908    1
2104    1
1180    0
591     0
1712    1
1761    0
259     0
2139    1
581     0
1165    0
74      1
885     0
273     1
824     1
812     1
598     0
263     0
Name: ats outcome, dtype: int64

In [179]:
list(range(0,len(training_data),2))[-1]

2252